In [1]:
import tensorflow as tf
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [19]:
tf.test.is_gpu_available()

False

In [20]:
batchsize=8

In [21]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'D:\DDS_Project\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'D:\DDS_Project\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 63119 images belonging to 2 classes.
Found 15779 images belonging to 2 classes.


In [22]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'D:\DDS_Project\Prepared_Data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 6000 images belonging to 2 classes.


In [23]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [24]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 39, 39, 32)           864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 39, 39, 32)           96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 39, 39, 32)           0         ['batch_normalization_94

In [25]:
from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [30]:
checkpoint = ModelCheckpoint(r'D:\DDS_Project\models\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [31]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

Epoch 1/5


C:\Users\MY HP\AppData\Local\Temp\ipykernel_4256\3918530525.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


7889/7889 [==============================] - ETA: 0s - loss: 0.1973 - accuracy: 0.9219
Epoch 1: val_loss improved from inf to 0.21330, saving model to D:\DDS_Project\models\model.h5


c:\Users\MY HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7889/7889 [==============================] - 701s 88ms/step - loss: 0.1973 - accuracy: 0.9219 - val_loss: 0.2133 - val_accuracy: 0.9097 - lr: 0.0010
Epoch 2/5
7889/7889 [==============================] - ETA: 0s - loss: 0.1790 - accuracy: 0.9306
Epoch 2: val_loss improved from 0.21330 to 0.18551, saving model to D:\DDS_Project\models\model.h5
7889/7889 [==============================] - 676s 86ms/step - loss: 0.1790 - accuracy: 0.9306 - val_loss: 0.1855 - val_accuracy: 0.9231 - lr: 0.0010
Epoch 3/5
7889/7889 [==============================] - ETA: 0s - loss: 0.1719 - accuracy: 0.9325
Epoch 3: val_loss did not improve from 0.18551
7889/7889 [==============================] - 612s 78ms/step - loss: 0.1719 - accuracy: 0.9325 - val_loss: 0.2026 - val_accuracy: 0.9185 - lr: 0.0010
Epoch 4/5
7889/7889 [==============================] - ETA: 0s - loss: 0.1665 - accuracy: 0.9367
Epoch 4: val_loss did not improve from 0.18551
7889/7889 [==============================] - 701s 89ms/step - loss: 0

In [ ]:
# Model Evaluation

In [32]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\MY HP\AppData\Local\Temp\ipykernel_4256\713691994.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.13639423251152039
0.9475277066230774


In [33]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\MY HP\AppData\Local\Temp\ipykernel_4256\4081756742.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.2016449123620987
0.9183091521263123


In [34]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

C:\Users\MY HP\AppData\Local\Temp\ipykernel_4256\3655471885.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.13639423251152039
0.9475277066230774
